In [1]:
'''
*** This is a slightly modified version of the original data processing script available with the dataset

This is the data processing script for POP909:A Pop song Dataset for Music Arrangement Generation
============
It will allow you to quickly process the POP909 Files (Midi) into the Google Magenta's music representation 
    as like [Music Transformer](https://magenta.tensorflow.org/music-transformer) 
            [Performance RNN](https://magenta.tensorflow.org/performance-rnn).

'''
import pickle
import os
import sys
from helpers.midi import MidiEventProcessor
import pretty_midi as pyd
import numpy as np
import tensorflow as tf
import pretty_midi
import librosa.display as ld
import matplotlib.pyplot as plt
from tqdm import tqdm

BASE_DIR = "/home/richhiey/Desktop/workspace/projects/virtual_musicians/data-store"
#BASE_DIR = "/home/richhiey/Desktop/workspace/projects/virtual_musicians"

#DATA_DIR = os.path.join(BASE_DIR, "data", "POP909-Dataset", "POP909")
DATA_DIR = os.path.join(BASE_DIR, "POP909-Dataset", "POP909")

#OUTPUT_DIR = os.path.join(BASE_DIR, "cache", "preprocessed", "POP909")
OUTPUT_DIR = os.path.join(BASE_DIR, "preprocessed")

MIDI_EVENTS_PATH = os.path.join(OUTPUT_DIR, "pop909-event-token.npy")
TFRECORD_DATASET = os.path.join(OUTPUT_DIR, "train.tfrecords")
AUGMENT_SCALE = 8

In [2]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

## -------------------------------------------------------------------
def plot_piano_roll(pm, start_pitch, end_pitch, fs=100):
    # Use librosa's specshow function for displaying the piano roll
    piano_roll = pm.get_piano_roll(fs)
    #print(piano_roll)
    plt.figure(figsize=(8, 4))
    ld.specshow(
        piano_roll[start_pitch:end_pitch],
        hop_length=1,
        sr=fs,
        x_axis='time',
        y_axis='cqt_note',
        fmin=pretty_midi.note_number_to_hz(start_pitch),
        fmax=pretty_midi.note_number_to_hz(end_pitch)
    )
    plt.colorbar()
    plt.title('Piano roll viz (notes)')
    plt.show()

def create_pm_object(notes):
    pm_outputs = pretty_midi.PrettyMIDI()
    # Create an Instrument instance for a cello instrument
    instr_prog = pretty_midi.instrument_name_to_program('Cello')
    instr = pretty_midi.Instrument(program=instr_prog)
    instr.notes = notes
    pm_outputs.instruments.append(instr)
    return pm_outputs

def prepare_midi_notes(notes, augment):
    for i in range(len(notes)):
        notes[i].start = round(notes[i].start,2)
        notes[i].pitch += augment
    notes.sort(key = lambda x:x.start)
    #print(notes)
    #plot_piano_roll(create_pm_object(notes), 40, 90)
    return notes

def preprocess_midi(midi_data, augment=0):
    separated_notes = {}
    repr_seq = {}
    mpr = MidiEventProcessor()

    for instr in midi_data.instruments:
        prepared_notes = prepare_midi_notes(instr.notes, augment)
        separated_notes[instr.name] = prepared_notes
        repr_seq[instr.name] = mpr.encode(separated_notes[instr.name])
        print('Augment value: ' + str(augment) + ' | Length of seq: ' + str(len(repr_seq[instr.name])))

    return repr_seq

def preprocess_pop909(midi_root, save_dir, augment_scale=0):
    save_py = []
    i = 0
    out_fmt = '{}-{}.data'
    midi_paths = [d for d in os.listdir(midi_root)]

    for path in tqdm(midi_paths):
        if (path.isnumeric()):
            i += 1
            filename = os.path.join(midi_root, path, path + '.mid')
            print('----------------------------------------------')
            print(i)
            print(' ', end='[{}]'.format(filename), flush=True)
            data = pyd.PrettyMIDI(filename)
            
            for augment_value in range(augment_scale + 1):
                try:
                    data_p = preprocess_midi(data, augment_value)
                    data_n = preprocess_midi(data, -1*augment_value)
                except KeyboardInterrupt:
                    print(' Abort')
                    return
                except EOFError:
                    print('EOF Error')
                    return
                save_py.append(data_p)
                save_py.append(data_n)
                
                
        
    save_py = np.array(save_py)
    print('Dataset length: ' + str(save_py.size))
    np.save(MIDI_EVENTS_PATH, save_py)
            
    
# replace the folder with your POP909 data folder
preprocess_pop909(DATA_DIR, OUTPUT_DIR, AUGMENT_SCALE)

  0%|          | 0/909 [00:00<?, ?it/s]

----------------------------------------------
1
 [/home/richhiey/Desktop/workspace/projects/virtual_musicians/data-store/POP909-Dataset/POP909/350/350.mid]Augment value: 0 | Length of seq: 2221
Augment value: 0 | Length of seq: 1302
Augment value: 0 | Length of seq: 3914
Augment value: 0 | Length of seq: 2221
Augment value: 0 | Length of seq: 1302
Augment value: 0 | Length of seq: 3914
Augment value: 1 | Length of seq: 2221
Augment value: 1 | Length of seq: 1302
Augment value: 1 | Length of seq: 3914
Augment value: -1 | Length of seq: 2221
Augment value: -1 | Length of seq: 1302
Augment value: -1 | Length of seq: 3914
Augment value: 2 | Length of seq: 2221

  0%|          | 1/909 [00:00<08:02,  1.88it/s]


Augment value: 2 | Length of seq: 1302
Augment value: 2 | Length of seq: 3914
Augment value: -2 | Length of seq: 2221
Augment value: -2 | Length of seq: 1302
Augment value: -2 | Length of seq: 3914
Augment value: 3 | Length of seq: 2221
Augment value: 3 | Length of seq: 1302
Augment value: 3 | Length of seq: 3914
Augment value: -3 | Length of seq: 2221
Augment value: -3 | Length of seq: 1302
Augment value: -3 | Length of seq: 3914
Augment value: 4 | Length of seq: 2221
Augment value: 4 | Length of seq: 1302
Augment value: 4 | Length of seq: 3914
Augment value: -4 | Length of seq: 2221
Augment value: -4 | Length of seq: 1302
Augment value: -4 | Length of seq: 3914
Augment value: 5 | Length of seq: 2221
Augment value: 5 | Length of seq: 1302
Augment value: 5 | Length of seq: 3914
Augment value: -5 | Length of seq: 2221
Augment value: -5 | Length of seq: 1302
Augment value: -5 | Length of seq: 3914
----------------------------------------------
2
 [/home/richhiey/Desktop/workspace/projec

  0%|          | 2/909 [00:00<07:07,  2.12it/s]

...

Augment value: -3 | Length of seq: 2245
Augment value: -3 | Length of seq: 986
Augment value: -3 | Length of seq: 4917
Augment value: 4 | Length of seq: 2245
Augment value: 4 | Length of seq: 986
Augment value: 4 | Length of seq: 4917
Augment value: -4 | Length of seq: 2245
Augment value: -4 | Length of seq: 986
Augment value: -4 | Length of seq: 4917
Augment value: 5 | Length of seq: 2245
Augment value: 5 | Length of seq: 986
Augment value: 5 | Length of seq: 4917
Augment value: -5 | Length of seq: 2245
Augment value: -5 | Length of seq: 986
Augment value: -5 | Length of seq: 4917
----------------------------------------------
908
 [/home/richhiey/Desktop/workspace/projects/virtual_musicians/data-store/POP909-Dataset/POP909/393/393.mid]Augment value: 0 | Length of seq: 1813
Augment value: 0 | Length of seq: 975
Augment value: 0 | Length of seq: 6350
Augment value: 0 | Length of seq: 1813
Augment value: 0 | Length of seq: 975
Augment value: 0 | Length of seq: 6350
Augment value: 1 | Le

100%|█████████▉| 908/909 [08:49<00:00,  1.55it/s]

Augment value: -2 | Length of seq: 6350
Augment value: 3 | Length of seq: 1813
Augment value: 3 | Length of seq: 975
Augment value: 3 | Length of seq: 6350
Augment value: -3 | Length of seq: 1813
Augment value: -3 | Length of seq: 975
Augment value: -3 | Length of seq: 6350
Augment value: 4 | Length of seq: 1813
Augment value: 4 | Length of seq: 975
Augment value: 4 | Length of seq: 6350
Augment value: -4 | Length of seq: 1813
Augment value: -4 | Length of seq: 975
Augment value: -4 | Length of seq: 6350
Augment value: 5 | Length of seq: 1813
Augment value: 5 | Length of seq: 975
Augment value: 5 | Length of seq: 6350
Augment value: -5 | Length of seq: 1813
Augment value: -5 | Length of seq: 975
Augment value: -5 | Length of seq: 6350
----------------------------------------------
909
 [/home/richhiey/Desktop/workspace/projects/virtual_musicians/data-store/POP909-Dataset/POP909/751/751.mid]Augment value: 0 | Length of seq: 1444
Augment value: 0 | Length of seq: 1837
Augment value: 0 | 

100%|██████████| 909/909 [08:50<00:00,  1.71it/s]

Augment value: 4 | Length of seq: 4207
Augment value: -4 | Length of seq: 1444
Augment value: -4 | Length of seq: 1837
Augment value: -4 | Length of seq: 4207
Augment value: 5 | Length of seq: 1444
Augment value: 5 | Length of seq: 1837
Augment value: 5 | Length of seq: 4207
Augment value: -5 | Length of seq: 1444
Augment value: -5 | Length of seq: 1837
Augment value: -5 | Length of seq: 4207
Dataset length: 10908
